In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import csv

nhl_page = requests.get("https://www.hockey-reference.com/leagues/NHL_2024_skaters-time-on-ice.html")
nhl_page_soup = BeautifulSoup(nhl_page.text, "html.parser")

all_nhl_skaters_table = nhl_page_soup.find('table', id='stats_toi')
if all_nhl_skaters_table is None:
    print('Not found')
else:                        ### Getting average even strength time on ice for defense and offense to better scale final numbers
    defensemen_toi = 0
    total_defensemen = 0
    offense_toi = 0
    total_offense = 0
    names = all_nhl_skaters_table.find_all('td', attrs = {'data-stat':'player'})
    games = all_nhl_skaters_table.find_all('td', attrs = {'data-stat':'games_played'})
    positions = all_nhl_skaters_table.find_all('td', attrs = {'data-stat':'pos'})
    toi = all_nhl_skaters_table.find_all('td', attrs = {'data-stat':'toi_ev'})
    for name, game, position, toi_i in zip(names, games, positions, toi):
        num = toi_i.text.replace(":", ".")
        if position.text != 'D':
            offense_toi += float(num)
            total_offense += 1
        else:
            defensemen_toi += float(num)
            total_defensemen += 1
        num = 0
    total_players = total_offense + total_defensemen
    avg_defensemen_toi = round(float(defensemen_toi) / float(total_defensemen), 2)
    avg_offense_toi = round(float(offense_toi) / float(total_offense), 2)
    final_str_defense_toi = str(avg_defensemen_toi).replace(".", ":")
    final_str_offense_toi = str(avg_offense_toi).replace(".", ":")

wsh_page = requests.get("https://www.hockey-reference.com/teams/WSH/2024.html#all_stats_adv_rs")
soup = BeautifulSoup(wsh_page.text, "html.parser")
advanced_table = soup.find('table', id='stats_toi')
if advanced_table is None:
    print('No advanced table found.')
else:
    wsh_names = advanced_table.find_all('td', attrs = {'data-stat':'player'})
    wsh_positions = advanced_table.find_all('td', attrs = {'data-stat':'pos'})
    wsh_games = advanced_table.find_all('td', attrs = {'data-stat':'games_played'})
    CF_rel_EV = advanced_table.find_all('td', attrs = {'data-stat':'corsi_rel_pct_ev'})
    TOI_EV =  advanced_table.find_all('td', attrs = {'data-stat':'toi_ev'})
    offense_weighted_toi = 0
    defense_weighted_toi = 0
    with open('CF%_TOI.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Player", "CF% Rel", "Weighted EV TOI"])
        for wsh_name, wsh_position, wsh_game, CF_rel_EV_i, TOI_EV_i in zip(wsh_names, wsh_positions, wsh_games, CF_rel_EV, TOI_EV):
            if int(wsh_game.text) < 10:   ### filters out players who played less than 10 games in 23-24 season
                pass
            else:
                if wsh_position.text != 'D':   ### checks if player is on offense or defense, because this matters when computing weighted EV TOI, since defensemen generally have more TOI than players on offense
                    offense_weighted_toi = round(float(TOI_EV_i.text.replace(":", ".")) / float(avg_offense_toi), 2) ### Computes weighted time on ice while even strength for player (player EV TOI/ Avg EV TOI of that player's position in the NHL)
                    writer.writerow([wsh_name.text, CF_rel_EV_i.text, offense_weighted_toi])
                    offense_weighted_toi = 0
                else:
                    defense_weighted_toi = round(float(TOI_EV_i.text.replace(":", ".")) / float(avg_defensemen_toi), 2)
                    writer.writerow([wsh_name.text, CF_rel_EV_i.text, defense_weighted_toi])
                    defense_weighted_toi = 0
file.close()
print('All done.')

All done.
